# Generación de Texto con modelos GPT

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Ohtar10/icesi-nlp/blob/main/Sesion5/1-text-generation.ipynb)

En este notebook harémos uso de un modelo tipo GPT-2 pre-entrenado en idioma español que utilizaremos para generar texto a partir de un contexto inicial que proveerémos. Luego, harémos fine tuning a este modelo con un dataset de chistes en español y observar como cambia la generación de texto en función del dataset que utilicemos.

#### Referencias
- Dataset: https://huggingface.co/datasets/mrm8488/CHISTES_spanish_jokes
- [Improving Language Understanding by Generative Pre-Training](https://cdn.openai.com/research-covers/language-unsupervised/language_understanding_paper.pdf)
- [Natural Language Processing with Transformers: Building Language Applications With Hugging Face](https://www.amazon.com/Natural-Language-Processing-Transformers-Applications/dp/1098103246)
- [GPT2 Spanish](https://huggingface.co/DeepESP/gpt2-spanish)
- [Fine-Tune a non-Englush GPT-2 Model with Huggingface](https://www.philschmid.de/fine-tune-a-non-english-gpt-2-model-with-huggingface)

In [1]:
import pkg_resources
import warnings

warnings.filterwarnings('ignore')

installed_packages = [package.key for package in pkg_resources.working_set]
IN_COLAB = 'google-colab' in installed_packages

/tmp/ipykernel_1215907/2396000874.py:1: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  import pkg_resources


In [ ]:
#!test '{IN_COLAB}' = 'True' && wget  https://github.com/Ohtar10/icesi-nlp/raw/refs/heads/main/requirements.txt && pip install -r requirements.txt
!test '{IN_COLAB}' = 'True' && sudo apt-get update -y
!test '{IN_COLAB}' = 'True' && sudo apt-get install python3.10 python3.10-distutils python3.10-lib2to3 -y
!test '{IN_COLAB}' = 'True' && sudo update-alternatives --install /usr/local/bin/python python /usr/bin/python3.11 2
!test '{IN_COLAB}' = 'True' && sudo update-alternatives --install /usr/local/bin/python python /usr/bin/python3.10 1
!test '{IN_COLAB}' = 'True' && pip install lightning datasets 'transformers[torch]' sentence-transformers tochinfo evaluate

## Generative pre-training Transformer - GPT

![](../assets/gpt.png)

Los modelos tipo GPT, introducidos por Radfor, et.al., de OpenAI, al igual que los modelos BERT, hacen uso extensivo de la arquitectura de transformers como hemos estado viendo. Las diferencias claves se podrían resumir en:

1. GPT utiliza bloques de **Transformer Decoder** encadenados, mientras que el modelo BERT utiliza bloques de *Transformer Encoder*
2. GPT se centra en la generación de texto basado en un contexto, la tarea principal es la predicción del siguiente token en la secuencia, mientras que BERT se centra en el completado de partes de una secuencia, en función de un contexto anterior y posterior a la secuencia de entrada. Entonces BERT se centra en la construicción de representación de lenguage, mientras que GPT se centra en la generación de texto en función de un contexto.

Sin embargo, ambos se basan en la misma premisa de pre-entrenar el modelo en tareas no-supervisadas o semi-supervisadas para que el modelo aprenda las representaciones semánticas del lenguage y luego al modelo se le pueda hacer fine tuning a tareas posteriores.

In [3]:
import torch
import transformers
from transformers import AutoModelForCausalLM, AutoTokenizer


device = "cuda" if torch.cuda.is_available() else "cpu"
model_name = "DeepESP/gpt2-spanish"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)
model

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [4]:
modules = [m for m, _ in model.named_modules()]
modules

['',
 'transformer',
 'transformer.wte',
 'transformer.wpe',
 'transformer.drop',
 'transformer.h',
 'transformer.h.0',
 'transformer.h.0.ln_1',
 'transformer.h.0.attn',
 'transformer.h.0.attn.c_attn',
 'transformer.h.0.attn.c_proj',
 'transformer.h.0.attn.attn_dropout',
 'transformer.h.0.attn.resid_dropout',
 'transformer.h.0.ln_2',
 'transformer.h.0.mlp',
 'transformer.h.0.mlp.c_fc',
 'transformer.h.0.mlp.c_proj',
 'transformer.h.0.mlp.act',
 'transformer.h.0.mlp.dropout',
 'transformer.h.1',
 'transformer.h.1.ln_1',
 'transformer.h.1.attn',
 'transformer.h.1.attn.c_attn',
 'transformer.h.1.attn.c_proj',
 'transformer.h.1.attn.attn_dropout',
 'transformer.h.1.attn.resid_dropout',
 'transformer.h.1.ln_2',
 'transformer.h.1.mlp',
 'transformer.h.1.mlp.c_fc',
 'transformer.h.1.mlp.c_proj',
 'transformer.h.1.mlp.act',
 'transformer.h.1.mlp.dropout',
 'transformer.h.2',
 'transformer.h.2.ln_1',
 'transformer.h.2.attn',
 'transformer.h.2.attn.c_attn',
 'transformer.h.2.attn.c_proj',
 'tran

Observermos un ejemplo de generación simple.

In [5]:
text = "Había una vez"
best = 10

with torch.no_grad():
    tokens = tokenizer(text, return_tensors='pt')['input_ids'].to(device)
    print("Dimensiones de la entrada:", tokens.shape)
    output = model(input_ids=tokens)
    print("Dimensiones de la salida:", output.logits.shape)
    output = output.logits[0, -1, :]
    print("Dimensiones del último token de la secuencia:", output.shape)
    probs = torch.softmax(output, dim=-1)
    print("Dimensiones de la probabilidad de los tokens:", probs.shape)
    sorted_probs = torch.argsort(probs, dim=-1, descending=True)
    print({tokenizer.decode(token): f"{prob.cpu().numpy() * 100:.2f}%" for token, prob in zip(sorted_probs[:best], probs[sorted_probs[:best]])})

Dimensiones de la entrada: torch.Size([1, 3])
Dimensiones de la salida: torch.Size([1, 3, 50257])
Dimensiones del último token de la secuencia: torch.Size([50257])
Dimensiones de la probabilidad de los tokens: torch.Size([50257])
{' más': '40.87%', ' que': '10.55%', ' en': '7.27%', ',': '4.90%', ' allí': '0.99%', ' se': '0.87%', '.': '0.82%', ' dentro': '0.81%', ' a': '0.78%', ' al': '0.78%'}


## Implementando una función de generación

Ahora, la idea es que este modelo nos sirva para generar texto de forma recurrente e incremental. En la última capa de los modelos tipo GPT encontrarémos un tensor con forma $(b, s, v)$, donde:

- $b$: Es el tamaño del bache, o la cantidad de secuencias a procesar.
- $s$: Es la longitud de la secuencia de entrada.
- $v$: Es el tamaño del vocabulario del modelo, cuantos tokens soporta.

Pero este es el tensor de salida, por qué tiene la forma de la secuencia de entrada?, porque cada posición en la salida corresponde a la la predicción del siguiente token de esa posición en la secuencia de entrada. En otras palabras, lo que obtenemos como predicción, es una secuencia de igual tamaño a la de entrada, movida un token hacia adelante, lo que efectivamente nos predice un solo token a la vez y por ende, el token que nos insteresa, es el último.

Lo que obtenemos en este tensor es además los logits de TODO el vocabulario del modelo, con los cuales podemos calcular las probabilidades de que cada uno sea el que continue en la secuencia. Hay varias formas de decodificar el siguiente token, la más fácil de implementar sería una decodificación codiciosa (greedy) del siguiente token, que consiste simplemente en seleccionar el token con la probabilidad más alta. Este es un enfoque simple y efectivo para algunos casos, pero al mismo tiempo sufre de poca variabilidad e incluso puede caer en generación repetitiva.

Otra opción es el muestreo, ya que justamente podemos obtener probabilidades del siguiente token, lo más lógico sería muestrear con esas opciones de probabilidad, de este modo podemos obtener mayor diversidad a la hora de generar el texto, al costo eso si de que haya mayor aleatoridad ya que se le daría la oportunidad a incluso tokens con baja probabilidad, de ser seleccionados.

Otra opción podría ser un balanceo entre una decodificación greedy y una por muestreo, en función de otro hiperparámetro que podemos definir. Esta sería una técnica muy común en el contexto de Reinforcement Learning llamade e-greedy. Se hace la aclaración de que en este ejemplo no harémos nada de RL, solamente se hace mención de esta técnica para balancear entre explotación y exploración.

In [6]:
import torch.nn as nn
import numpy as np
import pandas as pd
from typing import Optional, Tuple
from transformers.tokenization_utils_base import PreTrainedTokenizerBase


def generate(
        model: nn.Module, 
        tokenizer: PreTrainedTokenizerBase, 
        start: str, 
        max_length: int = 100, 
        eps: float = 0.5, 
        top_n: int = 5,
        return_iterations: bool = False,
        device: str = "cpu") -> Tuple[str, Optional[pd.DataFrame]]:

    output = [start]
    iterations = []
    with torch.no_grad():
        input_ids = tokenizer(output[-1], return_tensors='pt')['input_ids'].to(device)
        for _ in range(max_length):
            # Tomamos los logits producidos por la última capa del modelo
            # Estos corresponden al siguiente token por cada posición de la cadena
            logits = model(input_ids=input_ids).logits
            # Por lo tanto, el que nos interesa es el último, que correspondería a la
            # predicción del siguiente token después del final de la cadena original
            # A este aplicamos un softmax para obtener las probabilidades por cada
            # token del vocabulario para estar presente en la cadena.
            probs = torch.softmax(logits[0, -1, :], dim=-1)
            # Simplemente ordenamos por probabilidad de forma descendente
            sorted_tokens = torch.argsort(probs, dim=-1, descending=True)

            # Utilizamos una politica tipo e-greedy para obtener el siguiente token de la secuencia
            # Un eps>=1 quiere decir que siempre se va seleccionar el token de forma 'greedy', es decir
            # siempre se toma el token con probabilidad más alta.

            # Un eps=0 quiere decir que siempre se va a muestrear el siguiente token en función
            # de las probabilidades de cada token

            # Un 0<eps<1 va a balancear de forma binomial entre tomar el token con la
            # probabilidad más alta y muestrear el token en función de sus probabilidades. 
            if np.random.random_sample(1)[0] < eps:
                # Se toma el mejor token
                next_token = sorted_tokens[0].unsqueeze(dim=0)
            else:
                # Se muetrea el token de la probabilidad de distribución
                next_token = torch.multinomial(probs, 1)
            
            if return_iterations:
                # Mantenemos pista de todas las iteraciones para análisis
                iteration = {'input': ''.join(output)}
                best_n = sorted_tokens[:top_n].cpu().tolist()
                choices = {f'Choice #{choice+1}': f'{tokenizer.decode(token)} ({prob:.4f})' for choice, (token, prob) in enumerate(zip(best_n, probs[best_n].cpu().tolist()))}
                iteration.update(choices)
                iterations.append(iteration)

            output.append(tokenizer.decode(next_token))
            input_ids = torch.cat([input_ids, next_token.unsqueeze(dim=0)], dim=-1)

        output_text = ''.join(output)
        if not return_iterations:
            return output_text, None
        else:
            df = pd.DataFrame(iterations)
            return output_text, df

Ahora observemos que pasa cuando generamos texto con nuestra función y algunos parámetros.

Primero, observemos que pasa cuando pasamos un `eps=1` que quiere decir que la generación va a ser de tipo greedy:

In [7]:
output_text, iterations_df = generate(model, tokenizer, text, max_length=15, eps=1.0, top_n=10, return_iterations=True, device=device)
print(output_text)
iterations_df.head(15)

Había una vez más, el hombre que había sido su padre, el que había sido su


,input,Choice #1,Choice #2,Choice #3,Choice #4,Choice #5,Choice #6,Choice #7,Choice #8,Choice #9,Choice #10
0,Había una vez,más (0.4087),que (0.1055),en (0.0727),", (0.0490)",allí (0.0099),se (0.0087),. (0.0082),dentro (0.0081),a (0.0078),al (0.0078)
1,Había una vez más,", (0.5322)",en (0.0701),. (0.0519),la (0.0194),que (0.0191),: (0.0174),el (0.0162),se (0.0157),a (0.0142),y (0.0113)
2,"Había una vez más,",el (0.0491),se (0.0456),la (0.0409),en (0.0339),y (0.0301),no (0.0299),había (0.0205),me (0.0194),su (0.0158),los (0.0157)
3,"Había una vez más, el",hombre (0.0230),hecho (0.0225),señor (0.0173),joven (0.0165),recuerdo (0.0157),muchacho (0.0120),mundo (0.0119),corazón (0.0105),cuerpo (0.0095),deseo (0.0087)
4,"Había una vez más, el hombre",que (0.1364),se (0.0979),de (0.0590),había (0.0475),no (0.0358),le (0.0276),estaba (0.0216),la (0.0205),del (0.0204),al (0.0195)
5,"Había una vez más, el hombre que",había (0.1757),se (0.0684),estaba (0.0556),le (0.0475),la (0.0471),tenía (0.0348),lo (0.0310),me (0.0221),no (0.0146),amaba (0.0139)
6,"Había una vez más, el hombre que había",sido (0.0862),estado (0.0856),visto (0.0478),hablado (0.0323),conocido (0.0255),intentado (0.0232),hecho (0.0230),matado (0.0214),en (0.0184),entrado (0.0129)
7,"Había una vez más, el hombre que había sido",su (0.1692),", (0.0989)",el (0.0616),. (0.0441),antes (0.0383),en (0.0360),y (0.0337),capaz (0.0231),durante (0.0221),siempre (0.0183)
8,"Había una vez más, el hombre que había sido su",padre (0.1336),compañero (0.1296),amigo (0.1290),hermano (0.0368),marido (0.0357),primer (0.0353),hijo (0.0245),jefe (0.0181),mejor (0.0178),amante (0.0173)
9,"Había una vez más, el hombre que había sido su...",", (0.1805)",había (0.1334),. (0.1022),y (0.0761),se (0.0562),no (0.0505),en (0.0381),le (0.0301),durante (0.0280),era (0.0277)


Observamos como el input progresa a la vez que las opciones de tokens que hay. Sin importar cuantas veces invoquemos a la función con los mismos parámetros, siempre vamos a obtener los mismos resultados.

Ahora, observemos que pasa si introducimos exploración al reducir el `eps=0.5`, lo cual nos dice que aproximadamente la mitad de las veces va a elegir el siguiente token muestreando y la otra mitad explotando.

In [8]:
output_text, _ = generate(model, tokenizer, text, max_length=100, eps=0.5, device=device)
print(output_text)

Había una vez más, el hombre le mostró una sonrisa que prometía ser de hasta qué punto era alegre y una pizca de humor negro. 

Dolores cortó su conversación con Vicente. 

—No para nosotros, y de lo que no tenemos nada que decirnos es que no nos hemos pasado la vida en el campo. 

—No hay que pensar en nada. ¿Qué te parece si nos sentamos a cenar? 

—No, no, no tenemos nada que hacer —dijo divertida—. De hecho


En este caso, cada vez que invoquemos a la función, vamos a obtener una respuesta diferente, a veces más coherente y otras veces menos. Vale la pena realizar varias pruebas para observar los resultados hasta encontrar un balance.

### Generando texto con las utilidades del modelo

Ahora, la clase de Huggingface implementa la función `generate` que hace la labor de generación por nosotros, incluyendo las opciones de muestreo y explotación como hemos observado. Solo que además permite otra serie de parámetros y opciones para controlar la generación de texto. Para más información se recomienda estudiar:

- [Natural Language Processing with Transformers: Building Language Applications With Hugging Face](https://www.amazon.com/Natural-Language-Processing-Transformers-Applications/dp/1098103246), Capitulo 5
- https://huggingface.co/docs/transformers/v4.41.3/en/main_classes/text_generation#transformers.GenerationConfig
- https://huggingface.co/docs/transformers/v4.41.3/en/main_classes/text_generation#transformers.GenerationMixin.generate

In [9]:
output = model.generate(tokens, pad_token_id=tokenizer.eos_token_id, max_length=100, do_sample=True, temperature=0.5, top_k=0)
print(tokenizer.decode(output[0]))

Había una vez más, en aquel momento, un pensamiento se apoderó de él. 

—No es posible que yo hubiera sido tan afortunado —dijo. 

Pero no pudo contenerse. 

—Tú no eres el que se lo ha dicho a tu madre. 

—¿Por qué? 

—Porque ella se lo ha dicho a tu padre. 

—Pero ¿por qué? 

—Porque ella no lo ha dicho a tu madre. 

—¿Por qué?


## Fine tuning

Ahora, intentemos hacer fine tuning a nuestro modelo. Intentemos entrenarlo en un corpus de chistes en idioma español y ver como la narrativa de su output cambia.

##### Nota
Lastimosamente, este dataset es muy pequeño y la distribución del texto es muy diferente a la distribución de texto con la cual fue entrenado el modelo original, por lo que no se esperan resultados significativamente ejemplares. Sin embargo, el objetivo es observar como cambia la generación del texto una vez lo entrenamos en un conjunto especializado.

In [10]:
from datasets import load_dataset

dataset = load_dataset("mrm8488/CHISTES_spanish_jokes")
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'text', 'keywords', 'funny', 'category'],
        num_rows: 2419
    })
})

In [11]:
dataset['train'][0]

{'id': 0,
 'text': '- ¡Rápido, necesitamos sangre!\n- Yo soy 0 positivo.\n- Pues muy mal, necesitamos una mentalidad optimista.',
 'keywords': 'sangre',
 'funny': 1,
 'category': 'otros'}

In [12]:
dataset.set_format('pandas')
df = dataset['train'].to_pandas()
df.head(10)

,id,text,keywords,funny,category
0,0,"- ¡Rápido, necesitamos sangre!\n- Yo soy 0 pos...",sangre,1,otros
1,1,- ¿Cuál es el mejor portero del mundial? \n- E...,"futbol,porteros",1,otros
2,2,El otro día unas chicas llamarón a mi puerta y...,"dinero,agua",1,otros
3,3,"- Andresito, ¿qué planeta va después de Marte?...",planetas,1,profesiones
4,4,- ¿Por qué Bob Esponja no va al gimnasio? \n- ...,"esponja,gimnasios",1,otros
5,5,Van dos ciegos y le dice uno al otro: \n- Ojal...,ciegos,1,otros
6,6,Noticia de última hora!! \n\nMuere una suegra ...,"canarias,coches,noticias",2,familia
7,7,"– Mamá, mamá, en el colegio dicen que estoy lo...","locos,sillas",1,familia
8,8,"– Mamá, mamá, ¿me haces un bocata de jamón?\n–...","madres,jamón",1,otros
9,9,- Qué pasa si te expulsan de cuatro univerdade...,"universitarios,universidades",1,otros


In [13]:
df['Palabras por chiste'] = df['text'].str.split().apply(len)
df['Palabras por chiste'].median()

31.0

Aquí podemos observar que la mediana de longitud en terminos de palabras es de 31. Esto es esperado, pues los chistes deben ser cortos por naturaleza. Por otra parte, es bastante claro que el corpus original del modelo pre-entrenado contenía texto muy diferente a este, por lo que la calidad de los resultados, sin hacer mayores modificaciones puede que no sea buena.

Sin embargo, a manera demostrativa, continuarémos con el ejercicio, prepararémos el conjunto de datos para entrenamiento.

In [14]:
def preprocess_function(max_len):
    def _preprocess_function(examples):
        return tokenizer(examples['text'], max_length=max_len, truncation=True, padding='max_length')
    return _preprocess_function


Los modelos GPT no esperan otra cosa más que los `input_ids`, por lo que retirarémos todas las demás columnas del dataset ya que no nos son de utilidad en este momento. 

In [15]:
dataset.reset_format()
tokenized_dataset = dataset['train'].map(preprocess_function(max_len=512), batched=True)
tokenized_dataset = tokenized_dataset.remove_columns([col for col in tokenized_dataset.column_names if col != 'input_ids'])
tokenized_dataset = tokenized_dataset.train_test_split(train_size=0.9)
tokenized_dataset.set_format('torch')
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids'],
        num_rows: 2177
    })
    test: Dataset({
        features: ['input_ids'],
        num_rows: 242
    })
})

Finalmente procedemos a definir el entrenamiento. Observaremos que es muy similar a como entrenamos a BERT.

In [ ]:
from transformers import DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments


batch_size = 8 if IN_COLAB else 2
logging_steps = len(tokenized_dataset['train']) // batch_size
# Definimos los parámetros globales de entrenamiento
training_args = TrainingArguments(
    output_dir='./hf-gpt',
    overwrite_output_dir=True,
    num_train_epochs=10,
    learning_rate=2e-5,
    per_device_eval_batch_size=batch_size,
    per_device_train_batch_size=batch_size,
    weight_decay=0.01,
    eval_strategy='epoch',
    save_strategy='epoch',
    load_best_model_at_end=True,
    disable_tqdm=False,
    logging_steps=logging_steps,
    report_to='none'
)

# Y definimos el entrenador, especificando el modelo, datasets y el tokenizador
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False),
    train_dataset=tokenized_dataset['train'],
    eval_dataset=tokenized_dataset['test'],
    tokenizer=tokenizer
)

In [17]:
%%time
trainer.train()

Epoch,Training Loss,Validation Loss
1,3.465200,3.249673
2,2.918000,3.221956
3,2.573900,3.238452
4,2.303600,3.270826
5,2.080300,3.301478
6,1.904700,3.337730
7,1.758100,3.367514
8,1.651700,3.396965
9,1.581100,3.409228
10,1.527300,3.416162


There were missing keys in the checkpoint model loaded: ['lm_head.weight'].


CPU times: user 45min 52s, sys: 22.5 s, total: 46min 14s
Wall time: 46min 26s


TrainOutput(global_step=10890, training_loss=2.1758858590524497, metrics={'train_runtime': 2786.4402, 'train_samples_per_second': 7.813, 'train_steps_per_second': 3.908, 'total_flos': 5688327536640000.0, 'train_loss': 2.1758858590524497, 'epoch': 10.0})

Ahora observemos los resultados.

In [18]:
output = model.generate(tokens, pad_token_id=tokenizer.eos_token_id, max_length=100, do_sample=True, temperature=0.8)
print(tokenizer.decode(output[0]))

Había una vez un hombre conduciendo un coche por una carretera secundaria.
Al cabo de un rato el hombre le pregunta:
- ¿Con qué coche vive?
- Pues con el que yo quiero ir a ver.
Y el hombre le responde:
- ¡Ah, así que es eso!.
El hombre le pregunta:
- ¡Pues que si! ¡Pues que si!
Y el hombre le responde:
- ¡Pues que si! ¡Pues que si! ¡


In [19]:
output_text, _ = generate(model, tokenizer, text, max_length=100, eps=0.2, device=device)
print(output_text)

Había una vez un joven y un niño que se llamaba: 
- Oye yo, ¿dónde estás ahora?
- Vivo con mi madre. 
- Me alegro, pero mi madre se ha ido a vivir con mi hijo. 
- Genial, muy amable, pero no me ha dicho nada ni me ha dicho nada grave. 
- Vale, pero…yo lo divina, ¿dónde has estado a la hora de cenar? 
- Rafe Eeah. 
-


No parece ser muy gracioso precisamente, sin embargo, notemos que la generación de texto cambia de "estilo", ahora es mucho más frecuente encontrar conversaciones cortas, frases concisas, y situaciones particulares, en lugar del estilo más literario del modelo original. Esto es un indicio de la influencia que tiene el conjunto de datos de entrenamiento en el modelo final, esto es algo a tener muy en cuenta a la hora de utilizar y hacer fine tuning a modelos de lenguaje.

## Conclusiones
- Los modelos BERT y GPT son muy similares, aunque ambos tienen diferencias claras en cuanto a su estructura y manera de entrenamiento.
- Sin embargo, ambos pueden utilizarse para el mismo tipo de tareas posteriores, lo cual sustenta la importancia del pre-entrenamiento y la construcción de embeddings de buena calidad.
- En los modelos generativos, tiende a ver un dilema de tipo exploración-explitación, al explotar los resultados, podemos ser más precisos, per al mismo tiempo más monótonos, mientras que explorando podemos ser más creativos y diversos, pero al mismo tiempo terminar con texto incoherente, difuso o alucinante. Es necesario evaluar la tarea a la mano para escoger el ajuste adecuado entre estas dos técnicas de decodificación.
- Los modelos generativos de texto no son más que una gran probabilidad de distribución y esta a su vez es completamente dependiente de los datos con los que fue entrenada. Es aquí donde se hace sumamente importante obtener y curar los conjuntos de datos con los que se entrena, de lo contrario se puede terminar con un modelo de mala calidad para la tarea en especifico.
- Diferentes estrategias de decodificación entregan resultados diferentes, vale la pena hacer una exploración de los resultados y ajustar los hiperparámetros para obtener los resultados deseados según el objetivo.